In [ ]:
import time
import os
import numpy as np
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
from ipynb.fs.full.RoadUsersTrackingUtil import *
from ipynb.fs.full.TrafficLightDetection import *
from ipynb.fs.full.VideoProcessingUtil import *

def applyMaskToFrame(mog2BackgroundSubtractor, regionOfInterest):
    
    # Applying the mask on the region of interest using the Background Subtractor MOG2
    frameMask = mog2BackgroundSubtractor.apply(regionOfInterest)
    
    # Removing the non-white (gray) pixels from the shadows of the foreground objects present in the frame
    _, frameMask = cv2.threshold(frameMask, 254, 255, cv2.THRESH_BINARY)
    
    # Obtaining the co-ordinates of the moving objects (road users) present in the foreground of the frame
    contours, hierarchy = cv2.findContours(frameMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return frameMask,contours

def detectAndTrackRoadUsers(cameraStreamVideoFile, roiY1, roiYn, roiX1, roiXn):
    
    # Traffic lights detection constants
    TRAFFIC_LIGHTS_DNN_MODEL='../models/yolov3_training_final.weights'
    TRAFFIC_LIGHTS_DNN_CFG='../models/yolov3_testing.cfg'
    TRAFFIC_LIGHTS_DNN_CLASS_LABELS_FILE='../models/classes.txt'
    
    # Traffic enforcement camera constants
    PROCESSED_OUTPUT_VIDEOS_FOLDER='../processed_videos/'
    PROCESSED_OUTPUT_VIDEO_FILENAME2=os.path.join(PROCESSED_OUTPUT_VIDEOS_FOLDER,'2_Traffic_Lights_Detection.mp4')
    PROCESSED_OUTPUT_VIDEO_FILENAME3=os.path.join(PROCESSED_OUTPUT_VIDEOS_FOLDER,'3_Determining_Red_Light_Violations.mp4')
    OUTPUT_VIDEO_WIDTH=1920
    OUTPUT_VIDEO_HEIGHT=1080
    OUTPUT_VIDEO_FPS=30.26
    OUTPUT_VIDEO_FOURCC=VideoWriter_fourcc(*'MP42')
    
    # Preparing the video writer for generating the traffic lights detection video
    trafficLightsDetectionVideo = VideoWriter(PROCESSED_OUTPUT_VIDEO_FILENAME2, OUTPUT_VIDEO_FOURCC, float(OUTPUT_VIDEO_FPS), (OUTPUT_VIDEO_WIDTH, OUTPUT_VIDEO_HEIGHT))
    
    # Preparing the video writer for generating the red light violations video
    determiningRedLightViolationsVideo = VideoWriter(PROCESSED_OUTPUT_VIDEO_FILENAME3, OUTPUT_VIDEO_FOURCC, float(OUTPUT_VIDEO_FPS), (OUTPUT_VIDEO_WIDTH, OUTPUT_VIDEO_HEIGHT))
    
    # Loading the custom-made model (.weights) file and the configuration (.cfg) file
    deepNeuralNetwork=cv2.dnn.readNet(TRAFFIC_LIGHTS_DNN_MODEL,TRAFFIC_LIGHTS_DNN_CFG)
    
    # Creating a list of traffic light class labels
    trafficLights = []

    # Reading the traffic light class labels from the class labels file and storing it in the list for further processing
    with open(TRAFFIC_LIGHTS_DNN_CLASS_LABELS_FILE, "r") as classLabels:
        trafficLights = classLabels.read().splitlines()
    
    # Assigning the colour code randomly for the traffic light's bounding boxes
    colours = np.random.uniform(0, 255, size=(100, 3))
    
    # Using the Gaussian mixture-based background and foreground segmentation algorithm
    # to detect moving objects (road users) in the foreground against a stationary background (road)
    mog2BackgroundSubtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)
    
    # Creating a RoadUsersTrackingUtil class object for road user tracking
    roadUsersTrackingUtil = RoadUsersTrackingUtil()
    
    # Read the video stream generated by module 1
    cameraStreamVideo = VideoProcessingUtil(cameraStreamVideoFile).start()
    time.sleep(1.0)
    
    print('###### Module 2 - Traffic light detection and aspect tracking begins ######')
    print('RoadUsersDetectionAndTracking ---- Generating the traffic lights detection video . . .')
    
    # Reading the video stream frame-by-frame
    while (cameraStreamVideo.next()):
        frame = cameraStreamVideo.readQueue()
        if np.shape(frame) == ():
            break
        else:
            try:
                height, width, _ = frame.shape
            except AttributeError:
                print('RoadUsersDetectionAndTracking ---- Attribute error in the current frame while processing.')
                continue
            
            
            frame,redLightFlag=trafficLightDetectionAndTrack(frame,deepNeuralNetwork,trafficLights,colours)
            copiedFrame=frame.copy()
            trafficLightsDetectionVideo.write(copiedFrame)
            
            roadUserBoundingBoxes=[]
            
            # Setting the height and width for the traffic intersection as the region of interest
            regionOfInterest = frame[roiY1: roiYn,roiX1: roiXn]
            
            # Applying the mask to the frame using Background Subtractor MOG2 and region of interest
            frameMask,contours=applyMaskToFrame(mog2BackgroundSubtractor, regionOfInterest)
            
            for contour in contours:
                
                # Calculating the total area of the objects in the foreground so as to prevent the
                # detection and tracking of smaller components present in these objects such as wheels and helmets
                foregroundObjectArea = cv2.contourArea(contour)
                
                if foregroundObjectArea > 3000:
                    # Bound only the bigger, detected foreground objects
                    x, y, w, h = cv2.boundingRect(contour)
                    
                    # Append the values of the bounding box to a list for further processing
                    roadUserBoundingBoxes.append([x, y, w, h])
            
            # Fetching the bounding box ids for all the bounding boxes
            boundingBoxIds,crossedTrafficLightFlag = roadUsersTrackingUtil.getBoundingBoxIds(roadUserBoundingBoxes)
            
            # The primary logic to determine red light violation
            for boundingBoxId in boundingBoxIds:
                x, y, w, h, id = boundingBoxId
                
                # Check to see if the road user has crossed the stop line when the traffic light is red
                if(redLightFlag and crossedTrafficLightFlag):
                    print('RoadUsersDetectionAndTracking ---- Red light violation detected!')
                    cv2.putText(regionOfInterest, 'VIOLATOR ' + str(id), (x, y - 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
                    cv2.rectangle(regionOfInterest, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    redLightFlag=False
                    crossedTrafficLightFlag=False
                else:
                    cv2.putText(regionOfInterest, str(id), (x, y - 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 191, 0), 2)
                    cv2.rectangle(regionOfInterest, (x, y), (x + w, y + h), (255, 191, 0), 2)
            
            # Write the frame to the video
            determiningRedLightViolationsVideo.write(frame)
    
    print('RoadUsersDetectionAndTracking ---- The traffic lights detection video has been successfully generated.')
    print('###### Module 2 - Traffic light detection and aspect tracking ends ######')
    
    print('###### Module 3 - Road user detection and tracking begins ######')
    print('RoadUsersDetectionAndTracking ---- Generating the road user detection and tracking video . . .')
    
    print('RoadUsersDetectionAndTracking ---- The road user detection and tracking video has been successfully generated.')
    print('###### Module 3 - Road user detection and tracking ends ######')
    
    # Release all the resources
    determiningRedLightViolationsVideo.release()
    trafficLightsDetectionVideo.release()
    cameraStreamVideo.stop()
    cv2.destroyAllWindows()
    
    return;